In [ ]:
import numpy
import lvmdrp.core.resample as resample
import matplotlib.pyplot as plt
from scipy import interpolate
from astropy.io import fits

def interpolate_mask(x, y, mask, kind="linear", fill_value=0):
    """
    :param x, y: numpy arrays, samples and values
    :param mask: boolean mask, True for masked values
    :param method: interpolation method, one of linear, nearest,
    nearest-up, zero, slinear, quadratic, cubic, previous, or next.
    :param fill_value: which value to use for filling up data outside the
        convex hull of known pixel values.
        Default is 0, Has no effect for 'nearest'.
    :return: the image with missing values interpolated
    """
    if not numpy.any(mask):
        return y
    known_x, known_v = x[~mask], y[~mask]
    missing_x = x[mask]
    missing_idx = numpy.where(mask)

    f = interpolate.interp1d(known_x, known_v, kind=kind, fill_value=fill_value, bounds_error=False)
    yy = y.copy()
    yy[missing_idx] = f(missing_x)

    return yy



with fits.open("/Users/droryn/work/LVM/data/sas/sdsswork/lvm/spectro/redux/1.1.0dev/1039XX/1039949/60356/lvmCFrame-00012618.fits") as hdu:
    s1 = hdu['FLUX'].data
    d1 = hdu['IVAR'].data
    w1 = hdu['WAVE'].data

f = 1022
s1 = s1[f,:]
d1 = d1[f,:]

%matplotlib widget
# vega = numpy.genfromtxt('/Users/droryn/prog/sedfit/bc/A0V_KUR.SED', skip_header=1, usecols=(0,1))
# w1 = vega[:,0]
# lvm = numpy.where((w1>=3700)*(w1<9800))
# w1 = w1[lvm]
# s = vega[:,1][lvm]/1e8
fig = plt.figure(figsize=(12,5))
plt.plot(w1, s1, color='red', linewidth=1)
s1p = interpolate_mask(w1, s1, ~numpy.isfinite(s1))

w2 = numpy.logspace(numpy.log10(w1[1]), numpy.log10(w1[-1]), len(w1), endpoint=True)
# specutils
s2 = resample.rebin_spectra(w2, w1, s1p, spec_errs=None, fill=0)
plt.plot(w2, s2, color='blue', linewidth=1)
#%timeit rebin_spectra(w2, w1, s, spec_errs=None, fill=0)

# DESI cool histogram hack
s3 = resample.resample_flux(w2, w1, s1p, ivar=None)
#%timeit resample.resample_flux(w2, w1, s1)
plt.plot(w2, s3, color='cyan', linewidth=1)

# DESI projection matrix
s4 = resample.resample_project(w2, w1, s1p, ivar=None)
#%timeit resample.resample_flux(w2, w1, s1)
plt.plot(w2, s4, color='green', linewidth=1)

print(f'project: {numpy.nansum(s4*numpy.gradient(w2))}\n'\
      f'desifast: {numpy.nansum(s3*numpy.gradient(w2))}\n'\
      f'specutils: {numpy.nansum(s2*numpy.gradient(w2))}\n'\
      f'original: {numpy.nansum(s1p*numpy.gradient(w1))}')
#plt.plot(w2,s2, 'r.')